# Winequality dataset

## Feim els imports

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, Perceptron, RidgeCV, Lasso
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.feature_selection import SequentialFeatureSelector, SelectKBest, f_regression, RFE
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR
from feature_engine.creation import MathematicalCombination, CombineWithReferenceFeature

## Preparam el dataset

Funció que separa les features i les etiquetes, escalant les dades

In [ ]:
def load_data(df):
    y=df.quality
    X=df.drop('quality',axis=1)
    X = min_max_scaler.fit_transform(X)
    return X, y

Carregam els datasets a dos dataframes a partir dels fitxers csv: winequality-red.csv i winequality-white.csv

Afegim una nova columna que indiqui el tipus de vi a cada dataset i juntam els dos datasets en un de nou

In [ ]:
dfRed=pd.read_csv("winequality-red.csv", sep=";")
dfWhite=pd.read_csv("winequality-white.csv", sep=";")

dfRed['type']="Red"
dfWhite['type']="White"
df = pd.concat([dfRed,dfWhite])

Consultam les files del dataframe amb valors absents i les eliminam

In [ ]:
df[df.isna().any(axis=1)]
df.dropna(inplace=True)

Convertim la columna quality en una columna de valors categòrics

In [ ]:
dfAux=df.copy()
df.loc[dfAux['quality'] < 6, 'quality'] = "Baixa"
df.loc[dfAux['quality'] == 6, 'quality'] = "Mitjana"
df.loc[dfAux['quality'] > 6, 'quality'] = "Alta"
del dfAux

Convertim els valors categòrics en númerics

In [ ]:
labelencoder = preprocessing.LabelEncoder()

df['type']=labelencoder.fit_transform(df['type'])
df['quality']=labelencoder.fit_transform(df['quality'])

Recolocam les columnes

In [ ]:
df = df.reindex(columns=['fixed acidity', 'volatile acidity', 'citric acid', 
                       'residual sugar', 'chlorides', 'free sulfur dioxide',
                       'total sulfur dioxide', 'density', 'pH', 'sulphates', 
                       'alcohol', 'type', 'quality'])

In [ ]:
df.info()

In [ ]:
df.head()

Mesclam el dataframe i carregam les dades a les variables *features* (característiques) i *labels* (etiquetes).

A continuació podem observar les correlacions entre els distints atributs.

In [ ]:
min_max_scaler = preprocessing.MinMaxScaler()
df = df.sample(frac=1)

features, labels = load_data(df)

### Separació del conjunt d'entrenament i el de test
Per separar les dades en els conjunts d'entrenament i de test hem emprat la funció train_test_split

Hem decidit emprar un 80% entrenament i un 20% de test ja que  amb un valor més baix d'entrenament el model de regressió logistica i el del perceptró presentaven underfiting 


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

## Visualització de les dades

In [ ]:
np.set_printoptions(linewidth=300)

from IPython.display import display, HTML

display(HTML(data="""
<style>
    div#notebook-container    { width: 95%; }
    div#menubar-container     { width: 65%; }
    div#maintoolbar-container { width: 99%; }
</style>
"""))

corr = df.corr()

# Generam una màscara pel triangle superior
mask = np.triu(np.ones_like(corr, dtype=bool))

# Preparam el gràfic de pyplot
f, ax = plt.subplots(figsize=(11, 9))

# Dibuixam el mapa de calor amb la màscara i alguns paràmetres extra per millorar el resultat visual
sns.heatmap(corr, mask=mask, cmap="flare", square=True, linewidths=.25, cbar_kws={"shrink": .75})


Graficam el valor que prenen les característiques númeriques del dataframe mitjançant un histograma

In [ ]:
df.drop("type", axis=1).hist(figsize=(10,10), grid=False)

Gràfic cirular en el que podem veure la proporció de vi blanc i vermell del dataframe

In [ ]:
plt.pie(df['type'].value_counts(), labels = ["White","Red"], colors=["khaki","Maroon"])
print(df['type'].value_counts(normalize=True)*100)

# **Classificadors**

Per a cada model imprimim el seu classification report, la precisió de training i la de testing. A més, guardam aquests dos valors dins dos dataframes externs per després poder comparar les modificacions que farem al dataframe original.

Hem decidit fer els models dins funcions per poder cridar-les des de l'apartat d'[Enginyeria de característiques/Proves](#Proves) fora haver de reescriure el codi.

In [ ]:
dfTrainAccuracy = pd.DataFrame(columns=['Regressió Logística','Perceptró','Random Forest'])
dfTestAccuracy = pd.DataFrame(columns=['Regressió Logística','Perceptró','Random Forest'])

## Regressió Logística

Hem decidit donar-li valor al hiperparàmetre max_iters perquè amb el valor per defecte no arribava a convergir.

In [ ]:
def RegressioLogistica(df, nom):
    features, labels = load_data(df)
    X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)
    
    logreg = LogisticRegression(max_iter=5000)
    logreg.fit(X_train,y_train)

    y_predict = logreg.predict(X_test)
    

    print("\nRegressio Logística:")
    print(classification_report(y_test, y_predict,zero_division=1))
    
    training_accuracy = logreg.score(X_train,y_train)
    testing_accuracy = logreg.score(X_test,y_test)
    
    print('training accuracy:', training_accuracy*100)    
    print('testing accuracy:', testing_accuracy*100)
    
    #Afegim les dades a dos dataframes externs per després poder fer comparacions
    dfTrainAccuracy.at[nom,'Regressió Logística']=training_accuracy*100
    dfTestAccuracy.at[nom,'Regressió Logística']=testing_accuracy*100 
    
    print("\nConfusion Matrix")
    ConfusionMatrixDisplay(confusion_matrix(y_test,y_predict, labels=logreg.classes_), display_labels=logreg.classes_).plot()
    
    plt.show()

RegressioLogistica(df, "Original")

## Perceptró

In [ ]:
def Perceptro(df, nom):
    features, labels = load_data(df)
    X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)
    
    perceptro = Perceptron()
    perceptro.fit(X_train,y_train)

    y_predict = perceptro.predict(X_test)
    
    print("\nPerceptró:")
    print(classification_report(y_test, y_predict,zero_division=1))
    
    training_accuracy = perceptro.score(X_train,y_train)
    testing_accuracy = perceptro.score(X_test,y_test)
    
    print('training accuracy:', training_accuracy*100)    
    print('testing accuracy:', testing_accuracy*100)
    
    #Afegim les dades a dos dataframes externs per després poder fer comparacions
    dfTrainAccuracy.at[nom,'Perceptró']=training_accuracy*100
    dfTestAccuracy.at[nom,'Perceptró']=testing_accuracy*100 
    
    print("\nConfusion Matrix")
    ConfusionMatrixDisplay(confusion_matrix(y_test,y_predict, labels=perceptro.classes_), display_labels=perceptro.classes_).plot()
    
    plt.show()

Perceptro(df, "Original")

## Random Forest

Hem decidit donar-li valor al hiperparàmetre max_depth perquè amb el valor per defecte teniem overfitting.

In [ ]:
def RandomForest(df, nom):
    features, labels = load_data(df)
    X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)
    
    rf = RandomForestClassifier(max_depth=14)
    rf.fit(X_train, y_train)

    y_predict=rf.predict(X_test)

    print("\nRandom Forest:")
    print(classification_report(y_test, y_predict,zero_division=1))
    
    training_accuracy = rf.score(X_train,y_train)
    testing_accuracy = rf.score(X_test,y_test)
    
    print('training accuracy:', training_accuracy*100)    
    print('testing accuracy:', testing_accuracy*100)
    
    #Afegim les dades a dos dataframes externs per després poder fer comparacions
    dfTrainAccuracy.at[nom,'Random Forest']=training_accuracy*100
    dfTestAccuracy.at[nom,'Random Forest']=testing_accuracy*100
    
    print("\nConfusion Matrix")
    ConfusionMatrixDisplay(confusion_matrix(y_test,y_predict, labels=rf.classes_), display_labels=rf.classes_).plot()  
    plt.show()
           
RandomForest(df, "Original")

## Conclusions


Blablabla

# **Enginyeria de característiques**

L'enginyeria de característiques (feature engineering) és el procés de selecció, extracció, creació i transformació de les característiques d'un dataset amb l'objectiu de millorar l'eficàcia d'un model predictiu.

Per dur a terme l'enginyeria de característiques hem seguit els següents apartats: 
- **Feature importances dels models**: Ens permet saber la importància de cada característica dins cada model amb el fí d'eliminar les característiques menys importants.
- **Correlacions màximes i mínimes**: Ens permet saber quines característiques tenen major correlació entre elles. També ens permet saber quines característiques tenen menys correlació amb la qualitat del vi per després poder eliminar-les.
- **Feature Selection**: Selecció de les característiques més i menys importants. S'implementaran els següents:
   1. Sequential Feature Selection
   2. Univariate Feature Selection amb Kbest
   3. Recursive Feature Elimination

## Feature importances de cada model

In [ ]:
dfWquality=df.drop("quality", axis=1)

### Regressió logística <a id='Regressio-Logistica'></a>

In [ ]:
logreg = LogisticRegression(max_iter=5000)
logreg.fit(X_train,y_train)

logreg_odds = np.exp(logreg.coef_[0])
indices=np.argsort(logreg_odds)

fig, ax = plt.subplots()
ax.set_title("Feature importances for Logistic Regression")
ax.barh(range(len(logreg_odds)), logreg_odds[indices])
ax.set_yticks(range(len(logreg_odds)))
_ = ax.set_yticklabels(np.array(dfWquality.columns)[indices])

print("Millor característica:", dfWquality.columns[indices][-1])
pitjorLogReg =dfWquality.columns[indices][0]
print("Pitjor característica", pitjorLogReg)

### Perceptró <a id='Perceptro'></a>

In [ ]:
perceptron = Perceptron()
perceptron.fit(X_train, y_train)

perceptron_odds = np.exp(perceptron.coef_[0])
indices=np.argsort(perceptron_odds)

fig, ax = plt.subplots()
ax.set_title("Feature importances for Perceptron")
ax.barh(range(len(perceptron_odds)), perceptron_odds[indices])
ax.set_yticks(range(len(perceptron_odds)))
_ = ax.set_yticklabels(np.array(dfWquality.columns)[indices])

print("Millor característica:", dfWquality.columns[indices][-1])
pitjorPerc =dfWquality.columns[indices][0]
print("Pitjor característica:", pitjorPerc)

### Random Forest <a id='Random-Forest'></a>

In [ ]:
rf = RandomForestClassifier(max_depth=14)
rf.fit(X_train, y_train)

importancesRF = rf.feature_importances_
indices = np.argsort(importancesRF)

fig, ax = plt.subplots()
ax.set_title("Feature importances for Random forest")
ax.barh(range(len(importancesRF)), importancesRF[indices])
ax.set_yticks(range(len(importancesRF)))
_ = ax.set_yticklabels(np.array(dfWquality.columns)[indices])

print("Millor característica:", dfWquality.columns[indices][-1])
pitjorRF =dfWquality.columns[indices][0]
print("Pitjor característica:", pitjorRF)

### Comparativa entre models
| Model | Millor característica | Pitjor característica |
| ----------- | ----------- | ----------- |
| Regressió Logística | Alcohol | Volatile Acidity |
| Perceptró | Residual Sugar | Density |
| Random Forest | Alcohol | Type |

Es pot observar com l'alcohol dels vins pareix tenir una influència considerable en la qualitat d'aquests ja que, tant a la regressió logística com al random forest, és la millor característica i al perceptró es troba en segon lloc.

Els resultats poden variar depenent de l'execució, aquests valors són els obtinguts a una execució determinada.

## Correlacions màximes i mínimes <a id='Correlacions-Minimes'></a>

In [ ]:
c = df.corr().abs()
s = c.unstack()
so = s.sort_values(kind="quicksort", ascending=False)

print(so[13:23]) #Les 12 primeres son autocorrelacions
print("-------------")
print(so['quality'][10:13])
pitjorsCorrelacions=so['quality'][10:13].to_dict().keys()

corrs = df.corr().abs()['quality']
sig_cols = [col for col in corrs.keys() if col != 'quality' and abs(corrs[col]) > 0.1]
print(sig_cols)

Les característiques que tenen la major correlació amb la qualitat són el *Free sulfur dioxide* i el *Total sulfur dioxide*. \
Les característiques que tenen la menor correlació amb la qualitat són el *pH*, *residual sugars* i *sulphates*.

## [Feature Selection](https://www.scikit-learn.org/stable/modules/feature_selection.html)

### [Sequential Feature Selector](https://scikit-learn.org/stable/auto_examples/feature_selection/plot_select_from_model_diabetes.html#sphx-glr-auto-examples-feature-selection-plot-select-from-model-diabetes-py) <a id='Sequential-Feature-Selector'></a>


In [ ]:
feature_names = np.array(dfWquality.columns)
ridge = RidgeCV(alphas=np.logspace(-6, 6, num=5)).fit(X_train, y_train)

In [ ]:
sfs_forward = SequentialFeatureSelector(
    ridge, n_features_to_select=10, direction="forward"
).fit(X_train, y_train)


sfs_backward = SequentialFeatureSelector(
    ridge, n_features_to_select=10, direction="backward"
).fit(X_train, y_train)

pitjorSFSF=dfWquality.columns.difference(feature_names[sfs_forward.get_support()]).tolist()
pitjorSFSB=dfWquality.columns.difference(feature_names[sfs_backward.get_support()]).tolist()
print(type(pitjorSFSB))
print(
    "Features selected by forward sequential selection: "
    f"{feature_names[sfs_forward.get_support()]}"
    "\nFeatures not selected by forward sequential selection: "
    f"{pitjorSFSF}\n"
)

print(
    "Features selected by backward sequential selection: "
    f"{feature_names[sfs_backward.get_support()]}"
    "\nFeatures not selected by forward sequential selection: "
    f"{pitjorSFSB}"
)

### [Univariate Feature selection with Kbest](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.GenericUnivariateSelect.html#sklearn.feature_selection.GenericUnivariateSelect) <a id='Univariate-feature-selection'></a> 


In [ ]:
mask = SelectKBest(f_regression, k=10).fit(X_train, y_train).get_support()

best_features = dfWquality.columns[mask].tolist()
pitjorsUFS = dfWquality.columns.difference(best_features).tolist()

print("Millors característiques: ", best_features)
print("Pitjors característiques: ", pitjorsUFS)

### [Recursive Feature Elimination](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFE.html#sklearn.feature_selection.RFE) <a id='Recursive-feature-elimination'></a> 


In [ ]:
feature_names = np.array(dfWquality.columns)
ridge = RidgeCV(alphas=np.logspace(-6, 6, num=5)).fit(X_train, y_train)

In [ ]:
rfe_selector = RFE(ridge, n_features_to_select=10).fit(X_train, y_train)

pitjorsRFE=dfWquality.columns.difference(feature_names[rfe_selector.get_support()]).tolist()
print(
    "Característiques triades pel recursive feature elimination: "
    f"{feature_names[rfe_selector.get_support()]}"
    "\nCaracterístiques no triades pel recursive feature elimination: "
    f"{pitjorsRFE}\n"
)

### 
| Tipus | 1ra Pitjor Caract. | 2na Pitjor Caract.|
| ----------- | ----------- | ----------- |
| Sequential Feature Selection - Forward| Free sulfur dioxide | Total sulfur dioxide |
| Sequential Feature Selection - backward| Chloride | Citric Acid |
| Univariate Feature Selection | pH | Sulphates |
| Recursive Feature Elimination | Citric Acid | Type |

## Proves <a id='Proves'></a>

### Eliminam característiques

In [ ]:
dfComentaris = pd.DataFrame(columns=['Comentaris'])

#### Característica de menor importància de la Regressió Logística

Com s'ha observat [aquí](#Regressio-Logistica), el Feature Selection de la Regressió Logística indica que és la característica amb menys importància.

In [ ]:
df_menor_LogReg = df.drop(pitjorLogReg,axis=1)

RegressioLogistica(df_menor_LogReg, "menor imp. LogReg")
Perceptro(df_menor_LogReg, "menor imp. LogReg")
RandomForest(df_menor_LogReg, "menor imp. LogReg")

dfComentaris.at["menor imp. LogReg",'Comentaris']=pitjorLogReg

#### Característica de menor importància del Perceptró

Com s'ha observat [aquí](#Perceptro), el Feature Selection del Perceptró indica que és la característica amb menys importància.

In [ ]:
df_menor_Perc = df.drop(pitjorPerc,axis=1)

RegressioLogistica(df_menor_Perc, "menor imp. Perc")
Perceptro(df_menor_Perc, "menor imp. Perc")
RandomForest(df_menor_Perc, "menor imp. Perc")

dfComentaris.at["menor imp. Perc",'Comentaris']=pitjorPerc

#### Característica de menor importància del Random Forest

Com s'ha observat [aquí](#Random-Forest), el Feature Selection del Random Forest indica que és la característica amb menys importància.

In [ ]:
df_menor_RF = df.drop(pitjorRF,axis=1)

RegressioLogistica(df_menor_RF, "menor imp. RF")
Perceptro(df_menor_RF, "menor imp. RF")
RandomForest(df_menor_RF, "menor imp. RF")

dfComentaris.at["menor imp. RF",'Comentaris']=pitjorRF

#### Característica amb les pitjors correlacions amb la qualitat

Com s'ha observat [aquí](#Correlacions-Minimes), són les tres característiques menys correlacionades amb la qualitat.

In [ ]:
df_Correlacions = df.drop(pitjorsCorrelacions,axis=1)

RegressioLogistica(df_Correlacions, "Pitjors carac. Correlacions")
Perceptro(df_Correlacions, "Pitjors carac. Correlacions")
RandomForest(df_Correlacions, "Pitjors carac. Correlacions")

dfComentaris.at["Pitjors carac. Correlacions",'Comentaris']=pitjorsCorrelacions

#### Característica amb les pitjors característiques del Forward Feature Selection

Com s'ha observat [aquí](#Sequential-Feature-Selector), ja que el Forward Feature Selection indica que són les dues pitjors característiques.

In [ ]:
df_SFSF = df.drop(pitjorSFSF,axis=1)

RegressioLogistica(df_SFSF, "Pitjors carac. SFSF")
Perceptro(df_SFSF, "Pitjors carac. SFSF")
RandomForest(df_SFSF, "Pitjors carac. SFSF")

dfComentaris.at["Pitjors carac. SFSF",'Comentaris']=pitjorSFSF

#### Pitjors característiques Backward Feature Selection

Com s'ha observat [aquí](#Sequential-Feature-Selector), el Backward Feature Selection indica que són les dues pitjors característiques.

In [ ]:
df_SFSB = df.drop(pitjorSFSB,axis=1)

RegressioLogistica(df_SFSB, "Pitjors carac. SFSB")
Perceptro(df_SFSB, "Pitjors carac. SFSB")
RandomForest(df_SFSB, "Pitjors carac. SFSB")

dfComentaris.at["Pitjors carac. SFSB",'Comentaris']=pitjorSFSB

#### Pitjors característiques Univariate Feature Selection

Com s'ha observat [aquí](#Univariate-feature-selection), el SelectKBest indica que són les dues pitjors característiques.

In [ ]:
df_UFS = df.drop(pitjorsUFS,axis=1)

RegressioLogistica(df_UFS, "Pitjors carac. UFS")
Perceptro(df_UFS, "Pitjors carac. UFS")
RandomForest(df_UFS, "Pitjors carac. UFS")

dfComentaris.at["Pitjors carac. UFS",'Comentaris']=pitjorsUFS

#### Pitjors característiques Recursive Feature Selection

Com hem vist [aquí](#Recursive-feature-elimination), el Recursive Feature Elimination indica que són les dues pitjors característiques.

In [ ]:
df_RFE = df.drop(pitjorsRFE,axis=1)

RegressioLogistica(df_RFE, "Pitjors carac. RFE")
Perceptro(df_RFE, "Pitjors carac. RFE")
RandomForest(df_RFE, "Pitjors carac. RFE")

dfComentaris.at["Pitjors carac. RFE",'Comentaris']=pitjorsRFE

### Cream característiques

#### Àcids totals

In [ ]:
combinator_Acid_Tot = MathematicalCombination(
    variables_to_combine=['fixed acidity', 'volatile acidity'],
    math_operations = ['sum'],
    new_variables_names = ['total_acidity']
)

df_Acid_Tot = combinator_Acid_Tot.fit_transform(df)

RegressioLogistica(df_Acid_Tot, "Acid Totals")
Perceptro(df_Acid_Tot, "Acid Totals")
RandomForest(df_Acid_Tot, "Acid Totals")

#### Percentatge d'àcid cítric respecte de l'acidesa fixada

In [ ]:
combinator_PercAC = CombineWithReferenceFeature(
    variables_to_combine=['citric acid'],
    reference_variables=['fixed acidity'],
    operations=['div'],   
    new_variables_names=['percentage_citric_acid'])

df_PercAC = combinator_PercAC.fit_transform(df)

RegressioLogistica(df_PercAC, "Perc. acid citric")
Perceptro(df_PercAC, "Perc. acid citric")
RandomForest(df_PercAC, "Perc. acid citric")

#### Minerals totals

In [ ]:
combinator_Minerals_Totals = MathematicalCombination(
    variables_to_combine=['chlorides', 'sulphates'],
    math_operations = ['sum'],
    new_variables_names = ['total_minerals']
)

df_Minerals_Totals = combinator_Minerals_Totals.fit_transform(df)

RegressioLogistica(df_Minerals_Totals, "Minerals totals")
Perceptro(df_Minerals_Totals, "Minerals totals")
RandomForest(df_Minerals_Totals, "Minerals totals")

#### Percentatge de diòxid de sofre lliure

In [ ]:
combinator_PercSO2 = CombineWithReferenceFeature(
    variables_to_combine=['free sulfur dioxide'],
    reference_variables=['total sulfur dioxide'],
    operations=['div'],   
    new_variables_names=['percentage_free_sulfur'])

df_PercSO2 = combinator_PercSO2.fit_transform(df)

RegressioLogistica(df_PercSO2, "Perc. SO2 lliure")
Perceptro(df_PercSO2, "Perc. SO2 lliure")
RandomForest(df_PercSO2, "Perc. SO2 lliure")

#### Diòxid de sofre no lliure

In [ ]:
combinator_No_FreeSO2 = CombineWithReferenceFeature(
    variables_to_combine=['total sulfur dioxide'],
    reference_variables=['free sulfur dioxide'],
    operations=['sub'],
    new_variables_names=['non_free_sulfur_dioxide']
)

df_No_FreeSO2 = combinator_No_FreeSO2.fit_transform(df)

RegressioLogistica(df_No_FreeSO2, "S02 no lliure")
Perceptro(df_No_FreeSO2, "S02 no lliure")
RandomForest(df_No_FreeSO2, "S02 no lliure")


#### Dels diòxid de sofre quin percentatge és una sal

In [ ]:
combinator_Sulfurs_Sals = CombineWithReferenceFeature(
    variables_to_combine=['sulphates'],
    reference_variables=['free sulfur dioxide'],
    operations=['div'],
    new_variables_names=['percentage_salt_sulfur']
)

df_Sulfurs_Sals = combinator_Sulfurs_Sals.fit_transform(df)

RegressioLogistica(df_Sulfurs_Sals, "Sulfurs Sals")
Perceptro(df_Sulfurs_Sals, "Sulfurs Sals")
RandomForest(df_Sulfurs_Sals, "Sulfurs Sals")


#### Prova

In [ ]:
df_Prova = combinator_Acid_Tot.fit_transform(df)
df_Prova = combinator_PercSO2.fit_transform(df_Prova)

RegressioLogistica(df_Prova, "Prova")
Perceptro(df_Prova, "Prova")
RandomForest(df_Prova, "Prova")

#### Unim totes les creacions

In [ ]:
df_Unió = combinator_Acid_Tot.fit_transform(df)
df_Unió = combinator_Minerals_Totals.fit_transform(df_Unió)
df_Unió = combinator_PercAC.fit_transform(df_Unió)
df_Unió = combinator_No_FreeSO2.fit_transform(df_Unió)
df_Unió = combinator_PercSO2.fit_transform(df_Unió)
df_Unió = combinator_Sulfurs_Sals.fit_transform(df_Unió)

RegressioLogisitca(df_Unió, "Totes creacions")
Perceptro(df_Unió, "Totes creacions")
RandomForest(df_Unió, "Totes creacions")

### Comparam les precisions

In [ ]:
print("Training accuracy for each model and each dataframe\n", dfTrainAccuracy)
print("\nTesting accuracy for each model and each dataframe\n", dfTestAccuracy)
print("\n\nLes features eliminades han sigut", dfComentaris)

In [ ]:
dfDiffs = pd.DataFrame(columns=['Regressió Logística','Perceptró','Random Forest'])
for index, row in dfTestAccuracy.iterrows():
    dfDiffs.at[row.name]=row-dfTestAccuracy.loc['Original']
dfDiffs['Media'] = dfDiffs.mean(axis=1)
print(dfDiffs)

# **Grid Search CV**

In [ ]:
features, labels = load_data(df)
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

### Regressió Logística

In [ ]:
param_grid = {
    'penalty': ['l2', 'l1', 'elasticnet', 'none'],
    'C': [0.01, 0.1, 1, 10],
    'max_iter': [10,100,1000,2500,5000,7500,10000],
    'solver':['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
}
# Create a based model
logreg = LogisticRegression()
# Instantiate the grid search model
grid_search_logreg = GridSearchCV(estimator = logreg, param_grid = param_grid, cv = 3, n_jobs = -1, verbose = 2)
grid_search_logreg.fit(X_train, y_train)

print(grid_search_logreg.best_params_)

In [ ]:
logreg = LogisticRegression()
logreg.fit(X_train, y_train)

y_predict=logreg.predict(X_test)

print("\nLogistic Regression:")
print(classification_report(y_test, y_predict,zero_division=1))

training_accuracy = logreg.score(X_train,y_train)
testing_accuracy = logreg.score(X_test,y_test)

print('training accuracy:', training_accuracy*100)    
print('testing accuracy:', testing_accuracy*100)

print("\nConfusion Matrix")
ConfusionMatrixDisplay(confusion_matrix(y_test,y_predict, labels=logreg.classes_), display_labels=logreg.classes_).plot()  
plt.show()

#---------------------------------------------------------------------------

logregTunnedParams = logreg.set_params(**grid_search_logreg.best_params_)
logregTunnedParams.fit(X_train, y_train)

y_predict=logregTunnedParams.predict(X_test)

print("\nLogistic Regression:")
print(classification_report(y_test, y_predict,zero_division=1))

training_accuracy = logregTunnedParams.score(X_train,y_train)
testing_accuracy = logregTunnedParams.score(X_test,y_test)

print('training accuracy:', training_accuracy*100)    
print('testing accuracy:', testing_accuracy*100)

print("\nConfusion Matrix")
ConfusionMatrixDisplay(confusion_matrix(y_test,y_predict, labels=logregTunnedParams.classes_), display_labels=logregTunnedParams.classes_).plot()  
plt.show()

### Perceptró

In [ ]:
param_grid = {
    'penalty': ['l2', 'l1', 'elasticnet', 'none'],
    'alpha': [0.0001, 0.001, 0.01],
    'max_iter': [100,1000,2500,5000,7500,10000],
    'early_stopping': [True, False],
}
# Create a based model
perceptro = Perceptron()
# Instantiate the grid search model
grid_search_Perc = GridSearchCV(estimator = perceptro, param_grid = param_grid, cv = 3, n_jobs = -1, verbose = 2)
grid_search_Perc.fit(X_train, y_train)

print(grid_search_Perc.best_params_)

In [ ]:
perc = Perceptron()
perc.fit(X_train, y_train)

y_predict=perc.predict(X_test)

print("\nPerceptron:")
print(classification_report(y_test, y_predict,zero_division=1))

training_accuracy = perc.score(X_train,y_train)
testing_accuracy = perc.score(X_test,y_test)

print('training accuracy:', training_accuracy*100)    
print('testing accuracy:', testing_accuracy*100)

print("\nConfusion Matrix")
ConfusionMatrixDisplay(confusion_matrix(y_test,y_predict, labels=perc.classes_), display_labels=perc.classes_).plot()  
plt.show()

#---------------------------------------------------------------------------

percTunnedParams = perc.set_params(**grid_search_Perc.best_params_)
percTunnedParams.fit(X_train, y_train)

y_predict=percTunnedParams.predict(X_test)

print("\nPerceptron:")
print(classification_report(y_test, y_predict,zero_division=1))

training_accuracy = percTunnedParams.score(X_train,y_train)
testing_accuracy = percTunnedParams.score(X_test,y_test)

print('training accuracy:', training_accuracy*100)    
print('testing accuracy:', testing_accuracy*100)

print("\nConfusion Matrix")
ConfusionMatrixDisplay(confusion_matrix(y_test,y_predict, labels=percTunnedParams.classes_), display_labels=percTunnedParams.classes_).plot()  
plt.show()

### Random Forest

In [ ]:
param_grid = {
    'bootstrap': [True],
    'max_depth': [80, 90, 100, 110],
    'max_features': [2, 3],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100, 200, 300, 1000]
}
# Create a based model
rf = RandomForestClassifier(max_depth=14)
# Instantiate the grid search model
grid_search_rf = GridSearchCV(estimator = rf, param_grid = param_grid, cv = 3, n_jobs = -1, verbose = 2)
grid_search_rf.fit(X_train, y_train)

print(grid_search_rf.best_params_)


In [ ]:
y_predict=rf.predict(X_test)

print("\nRandom Forest:")
print(classification_report(y_test, y_predict,zero_division=1))

training_accuracy = rf.score(X_train,y_train)
testing_accuracy = rf.score(X_test,y_test)

print('training accuracy:', training_accuracy*100)    
print('testing accuracy:', testing_accuracy*100)

print("\nConfusion Matrix")
ConfusionMatrixDisplay(confusion_matrix(y_test,y_predict, labels=rf.classes_), display_labels=rf.classes_).plot()  
plt.show()

#---------------------------------------------------------------------------

rfTunnedParams = rf.set_params(**grid_search_rf.best_params_)
rfTunnedParams.fit(X_train, y_train)

y_predict=rfTunnedParams.predict(X_test)

print("\nRandom Forest:")
print(classification_report(y_test, y_predict,zero_division=1))

training_accuracy = rfTunnedParams.score(X_train,y_train)
testing_accuracy = rfTunnedParams.score(X_test,y_test)

print('training accuracy:', training_accuracy*100)    
print('testing accuracy:', testing_accuracy*100)

print("\nConfusion Matrix")
ConfusionMatrixDisplay(confusion_matrix(y_test,y_predict, labels=rfTunnedParams.classes_), display_labels=rfTunnedParams.classes_).plot()  
plt.show()
